## RDBMS data modeling

In [1]:
import psycopg2
from sql_queries import user_table_insert, fn_agg, ln_agg
import requests

In [2]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=zylotechdb user=student password=student")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
conn.set_session(autocommit=True)

In [3]:
# api-endpoint 
URL = "https://reqres.in/api/users?page="
#data requesting from endpoint pages
pages = [1, 2, 3, 4]

In [4]:
def check_user(id1):
    try: 
        cur.execute("SELECT * FROM users where user_id ='{0}'".format(id1))
        result = cur.fetchall()
        if len(result):
            return False
        else:
            return True
    except psycopg2.Error as e: 
        print("Error: select *")
        print (e)

In [5]:
for page in pages:
    URL = "https://reqres.in/api/users?page="+str(page)
    print(URL)
    try:
        res = requests.get(url = URL)
        res_json = res.json()
    except requests.exceptions.HTTPError as e:
        print(e)
    rows_affected = 0
    per_page = res_json['per_page']
    if res_json['data']:
        users = res_json['data']
        for user in users:
            id1 = user['id']
            email = user['email']
            first_name = user['first_name']
            last_name = user['last_name']
            avatar = user['avatar']
#                 print(id1, email, first_name, last_name, avatar)
            values = (id1,  first_name, last_name,email, avatar, "now()")
    
            try:
                if check_user(id1):
                    cur.execute(user_table_insert, values)
                    rows_affected += 1
                else:
                    print("Record already exists in the DB")
            except psycopg2.Error as e: 
                print("Error: Inserting Rows")
                print (e) 
        if rows_affected == per_page:
            print("Page = {0}, Status = All records inserted.".format(page))
        else:
            print("Page = {0}, Status = Missing  few records, per_page = {1}, Failed = {2}".format(page, per_page, (per_page-rows_affected)))
            
#         print("affected rows = {0}, per_page = {1}".format(rows_affected, per_page))
    else:
        print("No users list in the API end point.")


https://reqres.in/api/users?page=1
Page = 1, Status = All records inserted
affected rows = 3, per_page = 3
https://reqres.in/api/users?page=2
Page = 2, Status = All records inserted
affected rows = 3, per_page = 3
https://reqres.in/api/users?page=3
Page = 3, Status = All records inserted
affected rows = 3, per_page = 3
https://reqres.in/api/users?page=4
Page = 4, Status = All records inserted
affected rows = 3, per_page = 3


In [6]:
check_user(1)

False

In [7]:
try: 
    cur.execute("SELECT * FROM users;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    #print first_name and last_name
    print(row[2], row[3])
    row = cur.fetchone()

George Bluth
Janet Weaver
Emma Wong
Eve Holt
Charles Morris
Tracey Ramos
Michael Lawson
Lindsay Ferguson
Tobias Funke
Byron Fields
George Edwards
Rachel Howell


In [8]:
# Count of last name starting with same letter:
def aggregation_metrics(query):
    try: 
        cur.execute(query)
        rows = cur.fetchall()
        return rows
    except psycopg2.Error as e: 
        print("Error: ", query)
        print (e)

In [9]:
fn_agg_res = aggregation_metrics(fn_agg)
ln_agg_res = aggregation_metrics(ln_agg)

In [10]:
# try: 
#     cur.execute("select * from users")
#     rows = cur.fetchall()
#     print( rows)
# except psycopg2.Error as e: 
#     print("Error: ", query)
#     print (e)

In [11]:
for row in fn_agg_res:
    print(row)   

('B', 1)
('C', 1)
('E', 2)
('G', 2)
('J', 1)
('L', 1)
('M', 1)
('R', 1)
('T', 2)


In [12]:
for row in ln_agg_res:
    print(row)

('B', 1)
('E', 1)
('F', 3)
('H', 2)
('L', 1)
('M', 1)
('R', 1)
('W', 2)


## NoSQL data modeling

In [13]:
import cassandra
from cassandra.cluster import Cluster
from sql_queries import ac_users_table_create, ac_users_table_insert

In [14]:
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [15]:
try:
    session.set_keyspace('zylotechdb2')
except Exception as e:
    print(e)

In [16]:
try:
    session.execute(ac_users_table_create)
except Exception as e:
    print(e)

In [17]:
def insert_data(session, query):
    '''
        This function is used to insert records into tables
        args:
            session: holds connection
            query: string, query statement to insert into table.
        return: None
    '''
    for page in pages:
        URL = "https://reqres.in/api/users?page="+str(page)
        print(URL)
        try:
            res = requests.get(url = URL)
            res_json = res.json()
        except requests.exceptions.HTTPError as e:
            print(e)

        if res_json['data']:
            users = res_json['data']
            for user in users:
                id1 = user['id']
                email = user['email']
                first_name = user['first_name']
                last_name = user['last_name']
                avatar = user['avatar']
                session.execute(query, (int(id1), str(first_name), str(last_name), str(email), str(avatar)))

In [18]:

# INSERT into the table
insert_data(session, ac_users_table_insert)

https://reqres.in/api/users?page=1
https://reqres.in/api/users?page=2
https://reqres.in/api/users?page=3
https://reqres.in/api/users?page=4


In [19]:

def ac_aggregation_metrics(query):
    
    fc_dic = {}
    try:
        rows = session.execute(query)
        for row in rows:
#             print(row.first_char)
            fn_temp = row.first_char[0].upper()
            if fn_temp in fc_dic:
                fc_dic[fn_temp] +=1
            else:
                fc_dic[fn_temp] = 1
    except Exception as e:
        print(e)
    return fc_dic

In [20]:
#select query
fn_query = """SELECT  first_name as first_char
            FROM ac_users
            """
ln_query = """SELECT  last_name as first_char
            FROM ac_users
            """
fn_result = ac_aggregation_metrics(fn_query)
ln_result = ac_aggregation_metrics(ln_query)

In [21]:
print("fn:", fn_result)
print("ln:", ln_result)

fn: {'C': 1, 'B': 1, 'G': 2, 'L': 1, 'J': 1, 'E': 2, 'M': 1, 'T': 2, 'R': 1}
ln: {'M': 1, 'F': 3, 'E': 1, 'B': 1, 'W': 2, 'H': 2, 'L': 1, 'R': 1}


In [22]:
! pip install schedule

In [23]:
session.shutdown()
cluster.shutdown()